# Table of Contents
 <p><div class="lev1"><a href="#Warsztaty-modelowania-w-nanofizyce">Warsztaty modelowania w nanofizyce</a></div><div class="lev2"><a href="#Zachowania-atomów-w-zależności-od-ich-rodzaju-i-położenia">Zachowania atomów w zależności od ich rodzaju i położenia</a></div>

# Warsztaty modelowania w nanofizyce
----
## Zachowania atomów w zależności od ich rodzaju i położenia

**Paweł T. Jochym**

Zakład Komputerowych Badań Materiałów

Instytut Fizyki Jądrowej PAN, Kraków

----
Analiza przeprowadzona w ćwiczeniu [Amplitudy](Amplitudy.ipynb) wskazuje, że atomy w nanocząstce Pt-12Fe-42Pt zachowują się bardzo różnie w zależności od tego czy są to atomy żelaza czy platyny. Postarajmy się zbadać jak wygląda to zachowanie rysując struktury selekcjonując atomy jednego rodzaju. Zadanie selekcji wykona zmodyfikowana funkcja `recenter`.

In [1]:
# Import potrzebnych modułów
%matplotlib inline
import numpy as np
from ase import Atoms, units
import ase.io
from ase.io.trajectory import Trajectory
from IPython.core.display import Image
from ipywidgets import interact, interactive, fixed, IntRangeSlider, Dropdown, IntSlider
from ipywidgets import HBox, VBox, Checkbox, Dropdown, IntSlider, FloatSlider
from io import BytesIO
import pickle
import gzip
import nglview 
import glob
from time import sleep

In [2]:
mdtraj={}
for fn in glob.glob('data/md_T_*.traj'):
    # Użyj części nazwy pliku jako identyfikacji temperatury
    T=int(fn.split('/')[-1][5:-5])
    # Wczytaj trajektorię z pliku
    mdtraj[T]=Trajectory(fn)

# Wypisz wczytane temperatury
print(sorted(mdtraj.keys()))

[150, 300, 500, 600, 700, 800, 850, 900, 950, 1000, 1100, 1200, 1300, 1350, 1400, 1425, 1450, 1500, 1600, 1700, 2000]


In [3]:
def recenter(a, sel=None):
    '''
    Normalizacja położenia nanocząstki do stałej pozycji środka masy.
    Parametr sel pozwala wybrać atomy konkretnego pierwiastka.
    '''
    # Budowa tablicy logicznej służącej do maskowania listy atomów
    if sel is None :
        # Wszystkie pozycje True
        sel=(a.get_atomic_numbers()==a.get_atomic_numbers())
    else :
        # True w pozycjach atomów pierwiastka podanego w sel
        sel=(np.array(a.get_chemical_symbols())==sel)
        
    # Kopia struktury a
    c=Atoms(numbers=a.get_atomic_numbers()[sel],
            positions=a.get_positions()[sel],
            cell=a.get_cell(),
            pbc=a.get_pbc())
    
    # Przemieszczenie centrum masy do środka komórki
    # Uwaga: do działania konieczna jest istotna próżnia wokoło struktury
    c.translate((c.get_cell()/2).sum(axis=0)-c.get_center_of_mass())
    c.set_scaled_positions(c.get_scaled_positions())
    c.translate((c.get_cell()/2).sum(axis=0)-c.get_center_of_mass())
    c.set_scaled_positions(c.get_scaled_positions())
    return c

In [141]:
def show_cryst(struct, uc=True, re_center=True):
    view = nglview.show_ase(recenter(struct) if re_center else struct)
    view.parameters=dict(clipDist=-100)
    if uc : view.add_unitcell()
    view.camera='orthographic'
    view.add_spacefill(radiusType='covalent', scale=0.5)
    view.center()
    return view  

In [137]:
class TrajDisplay:
    def __init__(self, trajectories, re_center=False):
        if re_center :
            self.trajectories = {k:[recenter(a) for a in trj] 
                                     for k, trj in trajectories.items()}
        else :
            self.trajectories=trajectories
        self.temperatures=sorted(self.trajectories.keys())
        self.T=self.temperatures[0]
        self.trj=self.trajectories[self.T]

        self.v=nglview.show_asetraj(self.trj)
        self.v._remote_call("setSize", target="Widget", args=["500px", "500px"])
        self.v.add_spacefill()
        self.v.parameters=dict(clipDist=-100)
        self.v.camera='orthographic'
        self.v.update_spacefill(radiusType='covalent', scale=0.7)
        self.v.center()
        
        self.tsel=Dropdown(options=self.temperatures,
                           value=self.T, 
                           description='Temperatura:')
        self.frm=IntSlider(value=0, min=0, max=len(self.trj)-1)
        
        self.asel=Dropdown(options=list(set(self.trj[0].get_chemical_symbols()+['All'])),
                           value='All', description='Wyświetl')
        
        self.rad=FloatSlider(value=0.8, min=0.0, max=1.5, step=0.01, description='Rozmiar')

        self.tsel.observe(self._update_trj, 'value')
        self.frm.observe(self._update_frame)
        self.asel.observe(self._select_atom)
        self.rad.observe(self._update_repr)
        self._update_trj(None)
        
        self.hbox = HBox([self.v, VBox([self.tsel, 
                                        self.asel, 
                                        self.rad,
                                        self.frm])])

        
    def _update_repr(self, chg=None):
        self.v.update_spacefill(radiusType='covalent', scale=self.rad.value)        
        
    def _update_trj(self, chg=None):
        '''Zmiana trajektorii zależnie od wybranej temperatury'''
        self.T=int(self.tsel.value)
        self.trj=self.trajectories[self.T]
        self.frm.max = len(self.trj)-1
        v=self.v
        while v._ngl_component_ids :
            v.remove_component(v._ngl_component_ids[0])
        v.add_trajectory(nglview.ASETrajectory(self.trj))
        v.add_unitcell()
        v.center()
        self._select_atom()
        self._update_frame()
        return
    

    def _update_frame(self, chg=None):
        self.v.frame=self.frm.value
        return
    
    
    def _select_atom(self, chg=None):
        sel=self.asel.value
        self.v.remove_spacefill()
        if sel == 'All':
            self.v.add_spacefill(selection='All')
        else :
            self.v.add_spacefill(selection=[n for n,e in enumerate(self.trj[0].get_chemical_symbols()) if e == sel])
        self._update_repr()
            


In [144]:
show_cryst(mdtraj[1400][700])

In [138]:
dsp=TrajDisplay(ctrj,re_center=False)
dsp.hbox

In [ ]:
ctrj=dsp.trajectories